In [ ]:
import os
import sys
sys.path.append('/home/fruitspec-lab/FruitSpec/Code/fsCounter')
from omegaconf import OmegaConf
import pyzed.sl as sl
import cv2
import numpy as np
from scipy.stats import gaussian_kde
import kornia as K
from tqdm import tqdm
import numpy as np
import collections
import pandas as pd
from vision.misc.help_func import get_repo_dir, scale_dets, validate_output_path, scale
from vision.depth.zed.svo_operations import get_frame, get_depth, get_point_cloud, get_dimensions, sl_get_dimensions

repo_dir = get_repo_dir()
sys.path.append(os.path.join(repo_dir, 'vision', 'detector', 'yolo_x'))

from vision.pipelines.detection_flow import counter_detection
from vision.pipelines.misc.filters import filter_by_distance, filter_by_size, filter_by_height, sort_out
from vision.tracker.fsTracker.score_func import compute_dist_on_vec
from vision.data.results_collector import ResultsCollector
from vision.tools.translation import translation as T
from vision.tools.camera import is_sturated
from vision.tools.color import get_hue
from vision.tools.video_wrapper import video_wrapper
from vision.tools.image_stitching import plot_2_imgs
import matplotlib.pyplot as plt
import seaborn as sns
import kornia as K


In [ ]:
repo_dir = get_repo_dir()
pipeline_config = "/home/fruitspec-lab/FruitSpec/Code/fsCounter/vision/pipelines/config/pipeline_config.yaml"
runtime_config = "/home/fruitspec-lab/FruitSpec/Code/fsCounter/vision/pipelines/config/runtime_config.yaml"
cfg = OmegaConf.load(pipeline_config)
args = OmegaConf.load(runtime_config)

validate_output_path(args.output_folder)

In [ ]:
detector = counter_detection(cfg, args)
results_collector = ResultsCollector(rotate=args.rotate)
translation = T(cfg.translation.translation_size, cfg.translation.dets_only, cfg.translation.mode)

cam = video_wrapper(args.movie_path, args.rotate, args.depth_minimum, args.depth_maximum)

In [ ]:
measures_path = "/home/fruitspec-lab/Downloads/tomato/analysis/230123/pre/5/measures_pix_size_mean_hue_depth.csv"
measures_frame = pd.read_csv(measures_path)

In [ ]:
def plot_image_grid(images, nrows, ncols, titles=None, figsize=None, xlabels=None, ylabels=None, cmap='viridis'):
    """
    Plots a grid of images using matplotlib.

    Parameters:
    images (list): A list of numpy arrays representing the images to be plotted.
    nrows (int): The number of rows in the grid.
    ncols (int): The number of columns in the grid.
    titles (list, optional): A list of strings representing the titles of the images. Must have the same length as images.
    figsize (tuple, optional): A tuple representing the size of the figure. Defaults to (ncols * 5, nrows * 5).
    xlabels (list, optional): A list of strings representing the x-axis labels for each image. Must have the same length as images.
    ylabels (list, optional): A list of strings representing the y-axis labels for each image. Must have the same length as images.
    cmap (str, optional): The color map to use when plotting the images. Defaults to 'viridis'.

    Returns:
    None
    """
    n_images = len(images)
    if isinstance(figsize, type(None)):
        figsize = (ncols * 5, nrows * 5)
    titles, xlabels, ylabels = (np.full(n_images, "") if isinstance(titles, type(None)) else arr
                                for arr in [titles, xlabels, ylabels])
    fig, axs = plt.subplots(nrows, ncols, figsize=figsize)
    for i, ax in enumerate(axs.flat):
        image = images[i]
        ax.imshow(image)
        ax.set_title(titles[i])
        ax.set_xlabel(xlabels[i])
        ax.set_ylabel(ylabels[i])
    plt.tight_layout()
    plt.show()


def cut_center_of_box(image, margin=0.05):
    """
    Cuts the center of the box.
    
    Args:
    - image: A 3D Numpy array representing a cropped xyz image.
    - margin: A float representing the percentage of margin to remove from the center of the image.

    Returns:
    - A 3D Numpy array representing the cropped image with the outside of the box removed.
    """
    t, l, (b, r) = 0 ,0, image.shape[:2]
    y_max, x_max = image.shape[:2]
    h_m = int((b-t)*margin)
    w_m = int((r-l)*margin)
    cut_box = image[max(0, t+h_m):min(y_max, b-h_m), max(0, l+w_m):min(x_max, r-w_m)]
    return cut_box


def xyz_center_of_box(image, method="median"):
    """
    Calculates the median or mean x, y, and z coordinates of the fruit.

    Args:
    - image: A 3D Numpy array representing a cropped xyz image.
    - method: A string representing the method to use to calculate the center of the fruit. Must be either "median" or "mean".

    Returns:
    - A tuple of floats representing the x, y, and z coordinates of the center of the fruit.
    """
    if method=="median":
        cut_box = cut_center_of_box(image, 0.025)
        x_median = np.nanmedian(cut_box[:, :, 0])
        y_median = np.nanmedian(cut_box[:, :, 1])
        z_median = np.nanmedian(cut_box[:, :, 2])
    elif method=="mean":
        cut_box = cut_center_of_box(image, 0.4)
        x_median = np.nanmean(cut_box[:, :, 0])
        y_median = np.nanmean(cut_box[:, :, 1])
        z_median = np.nanmean(cut_box[:, :, 2])
    else: # calculates only on the edge of the cut box
        cut_box = cut_center_of_box(image, 0.25).copy()
        if cut_box.shape[0] > 10 and cut_box.shape[1] > 10:
            cut_box[5:-5,5:-5] = np.nan
        x_median = np.nanmedian(cut_box[:, :, 0])
        y_median = np.nanmedian(cut_box[:, :, 1])
        z_median = np.nanmedian(cut_box[:, :, 2])
    return x_median, y_median, z_median


def dist_to_box_center(image, method="median"):
    """
    Calculates the distance from the camera to the center of the fruit.

    Args:
    - image: A 3D Numpy array representing a cropped xyz image.
    - method: A string representing the method to use to calculate the center of the fruit. Must be either "median" or "mean".

    Returns:
    - A float representing the distance from the camera to the center of the fruit.
    """
    return np.sum(np.array(list(xyz_center_of_box(image, method)))**2)

def depth_to_box_center(image, method="median"):
    """
    Calculates the depth from the camera to the center of the fruit.

    Args:
    - image: A 3D Numpy array representing a cropped xyz image.
    - method: A string representing the method to use to calculate the center of the fruit. Must be either "median" or "mean".

    Returns:
    - A float representing the depth from the camera to the center of the fruit.
    """
    return xyz_center_of_box(image, method)[2]


def hue_filtering(rgb_crop, nstds = 1):
    """
    Apply hue filtering to an RGB image crop.

    Parameters:
        rgb_crop (numpy.ndarray): Input RGB image crop as a numpy array.
        nstds (float): Number of standard deviations used to determine the upper and lower hue thresholds.
    
    Returns:
        numpy.ndarray: mask where True value indicated what was filtered out
    """
    rgb_c = rgb_crop.copy()
    hsv = cv2.cvtColor(rgb_c, cv2.COLOR_RGB2HSV)
    hue, sat, v = cv2.split(hsv.copy())
    hist_vals, hist_bins = np.histogram(hue, bins = 50)
    mode = hist_bins[np.argmax(hist_vals)]
    if mode > 35: # greener area
        nstds *= 1.5
    hue_std = np.std(hue)
    upper_limit = mode + nstds*hue_std
    lower_limit = mode - nstds*hue_std
    logical_vec = np.any([hue > upper_limit, hue < lower_limit], axis = 0)
    return logical_vec

In [ ]:
def get_track_id_frames(measures_frame, track_id):
    """
    Returns a numpy array of frames associated with a given track ID.

    Parameters:
    measures_frame (pandas.DataFrame): A dataframe containing measurements data.
    track_id (int): The ID of the track to retrieve frames for.

    Returns:
    numpy.ndarray: A 1-dimensional array of frames associated with the given track ID.
    """
    return measures_frame[measures_frame["track_id"] == track_id]["frame"].values

def get_track_id_boxes(measures_frame, track_id):
    """
    Returns a numpy array of bounding boxes associated with a given track ID.

    Parameters:
    measures_frame (pandas.DataFrame): A dataframe containing measurements data.
    track_id (int): The ID of the track to retrieve bounding boxes for.

    Returns:
    numpy.ndarray: A 2-dimensional array of bounding boxes associated with the given track ID.
        Each row contains the x1, y1, x2, y2 coordinates of a bounding box in the format [x1, y1, x2, y2].
    """
    return measures_frame[measures_frame["track_id"] == track_id][["x1", "y1", "x2", "y2"]].values

def validate_bbox(crop, rgb_img):
    """
    Validates the given bounding box coordinates and ensures that they fall within the dimensions of the RGB image.

    Parameters:
    crop (tuple): A tuple containing the coordinates of the bounding box in the format (x1, y1, x2, y2).
        x1 and y1 are the coordinates of the top-left corner of the bounding box, and x2 and y2 are the coordinates of the bottom-right corner.
    rgb_img (numpy.ndarray): A numpy array representing the RGB image.

    Returns:
    tuple: A tuple containing the validated bounding box coordinates in the format (x1, y1, x2, y2).
        The returned coordinates ensure that the bounding box falls entirely within the dimensions of the RGB image.
    """
    x1, y1, x2, y2 = crop
    x1 = max(x1, 0)
    y1 = max(y1, 0)
    h, w = rgb_img.shape[:2]
    x2 = min(x2, w-1)
    y2 = min(y2, h-1)
    return x1, y1, x2, y2


def get_track_id_images(measures_frame, track_id, args):
    cam = video_wrapper(args.movie_path, args.rotate, args.depth_minimum, args.depth_maximum)
    frame_numbers = get_track_id_frames(measures_frame, track_id)
    boxes = get_track_id_boxes(measures_frame, track_id)
    rgb_images, pc_images = [], []
    for i in tqdm(range(len(frame_numbers))):
        frame, crop = frame_numbers[i], boxes[i]
        rgb_img, _, pc_img = cam.get_zed(frame)
        x1,y1,x2,y2 = validate_bbox(crop, rgb_img)
        rgb_images.append(rgb_img[y1:y2,x1:x2, ::-1])
        pc_images.append(pc_img[y1:y2,x1:x2])
    cam.close()
    return rgb_images, pc_images

def kde_filtering(centers, thresh=0.5):
    """
    Applies a Kernel Density Estimation (KDE) filtering on a set of 3D points.

    Args:
        centers (np.ndarray): A numpy array of shape (n, 3) representing the 3D coordinates of the points to filter.
        thresh (float): A threshold value to filter out points with low density. Default is 0.5.

    Returns:
        np.ndarray: A numpy array of shape (n, 3) where each filtered 3D points is replaced with np.nan
    """
    finite_logical = np.all(np.isfinite(centers), axis=1)
    if not sum(finite_logical):
        return centers
    finite_centers = centers[finite_logical].copy()
    kernel = gaussian_kde(finite_centers.T)
    points_density = np.full(len(centers), np.nan)
    points_density[finite_logical] = kernel(finite_centers.T)
    points_density = points_density/np.nansum(points_density)
    filtered_center = centers.copy()
    filtered_center[points_density <thresh/ len(finite_logical)] = np.nan
    return filtered_center

In [ ]:
# rgb_images, pc_images = get_track_id_images(measures_frame, 7)
# plot_2_imgs(rgb_images[0], pc_images[0][:,:,2])

In [ ]:
def filter_xyz_outliers(crop, nstd=2, as_points=True):
    """
    Filters out the outliers from the 3D points in the given crop.

    Args:
        crop (ndarray): A numpy array of shape (height, width, 3) containing the 3D points.
        nstd (float): The number of standard deviations to consider for defining the range of valid values.
        as_points (bool): Whether to return the filtered 3D points as an array of points or as an array of the same shape
            as the input crop.

    Returns:
        ndarray: A numpy array of filtered 3D points. If as_points is True, this is a numpy array of shape (n, 3),
            where n is the number of valid 3D points. Otherwise, it is a numpy array of the same shape as the input crop.
    """
    centers = crop.reshape(-1, 3)
    filtered_centers = centers.copy()
    channel_medians = np.nanmedian(centers, axis=0)
    channel_stddevs = np.nanstd(centers, axis=0)
    channel_max = channel_medians + nstd * channel_stddevs
    channel_min = channel_medians - nstd * channel_stddevs
    valid_mask = np.all((centers >= channel_min) & (centers <= channel_max), axis=1)
    filtered_centers[~valid_mask] = np.nan
    if as_points:
        return filtered_centers
    return filtered_centers.reshape(crop.shape)


def ellipsoid_fit(filtered_center):
    """
    Fits an ellipsoid to a set of 3D points using least squares estimation.

    Args:
    - filtered_center (numpy array): An N x 3 array of N 3D points in the form (x, y, z).

    Returns:
    - radius (float): The radius of the fitted ellipsoid.
    - semi_axis_1 (float): The length of the semi-major axis of the ellipsoid.
    - semi_axis_2 (float): The length of the semi-intermediate axis of the ellipsoid.
    - semi_axis_3 (float): The length of the semi-minor axis of the ellipsoid.
    """
    filtered_center[np.abs(filtered_center) > 2] = np.nan
    A = np.column_stack([filtered_center, np.ones(len(filtered_center))])
    good_rows = np.all(np.isfinite(A), axis=1)
    A = A[good_rows]

    #   Assemble the f matrix
    f = np.zeros((len(A),1))
    f[:,0] = np.sum(A[:,:3]**2, axis = 1)
    C, residules, rank, singval = np.linalg.lstsq(A,f)
    C = np.abs(C)
    #   solve for the radius
    t = (C[0]*C[0])+(C[1]*C[1])+(C[2]*C[2])+C[3]
    radius = np.sqrt(t)
    # channels are switched
    return radius, np.sqrt(C[1]), np.sqrt(C[0]), np.sqrt(C[2])

In [ ]:
from sklearn.linear_model import RANSACRegressor
def get_cropped_point_cloud(bbox, point_cloud, margin=0.2):
    crop = point_cloud[max(int(bbox[1]), 0):int(bbox[3]), max(int(bbox[0]), 0): int(bbox[2]), :-1].copy()
    return crop

In [ ]:
from vision.visualization.drawer import draw_rectangle, draw_text, draw_highlighted_test, get_color
def get_color(rgb_crop):
    if not len(rgb_crop):
        return (255,255,255)
    hsv = cv2.cvtColor(rgb_crop.astype(np.uint8), cv2.COLOR_RGB2HSV)
    hue, sat, val = cv2.split(hsv.copy())
    mean_hue, median_hue = np.nanmean(hue), np.nanmedian(hue)
    hist_vals, hist_bins = np.histogram(hue, bins = 100)
    mode_hue = hist_bins[np.argmax(hist_vals)]
    if mode_hue < 10:
        return (255,0,0)
    if mode_hue < 17.5:
        return (255,125,0)
    if mode_hue < 45:
        w, h = rgb_crop.shape[:2]
        w_025 = int(w/4)
        h_025 = int(h/4)
        hue_cut = hue[h_025:-h_025,w_025:-w_025]
        mean_hue_cut, median_hue_cut = np.nanmean(hue_cut), np.nanmedian(hue_cut)
        if median_hue_cut * 1.15 > mean_hue_cut: # low "skew"
            return (255,255,0)
        else:
            return (128,0,128)
    return (0,255,0)


def draw_dets(frame, dets, t_index=6):
    out_frame = frame.copy()
    for det in dets:
        x1,y1,x2,y2 = validate_bbox(det[:4].astype(np.int), frame)
        track_id = det[t_index]
        color_id = int(track_id) % 15  # 15 is the number of colors in list
        color = get_color(frame[y1:y2, x1:x2])
        text_color = (0,0,0)
        out_frame = draw_rectangle(out_frame, (int(det[0]), int(det[1])), (int(det[2]), int(det[3])), color, 3)
        out_frame = draw_highlighted_test(out_frame, f'ID:{track_id}', (det[0], det[1]), frame.shape[1],
                                          color, text_color, True, 10, 3)

    return out_frame
cam = video_wrapper(args.movie_path, args.rotate, args.depth_minimum, args.depth_maximum)
for frame_number in range(50,800, 5):
    rgb_img, _, pc_img = cam.get_zed(frame_number)
    if isinstance(rgb_img, type(None)):
        continue
    rgb_img = rgb_img[:,:,::-1]
    boxes_for_frame = measures_frame[measures_frame["frame"]==frame_number].values
    plt.figure(figsize = (10,10))
    plt.imshow(draw_dets(rgb_img.astype(np.uint8), boxes_for_frame)[900:-300,:,:])
    plt.title(frame_number)
    plt.show()
cam.close()

In [ ]:
# green_fruits = [53, 71, 131 ,108, 91, 228, 225 ,247, 442, 452, 502,943]
# green_fruits_sat = [61,70]
# breaking_fruits = [227, 163, 217, 851, 260, 304,1220, 1752, 304,450,2397]
# yellow_fruits = [82, 95, 739 ,856, 959, 1149, 1530,1645]
# oragne_fruits = [246, 451, 410, 1112, 1235, 1464,1525, 1911, 2298, 1464]
# red_fruits = [90, 558,515, 1687, 1706, 1910, 558,1129, 1515, 1778,1905, 2394]

# fruit_dict = {
#     "red": red_fruits,
#     "orange": oragne_fruits,
#     "yellow": yellow_fruits,
#     "breaking": breaking_fruits,
#     "green": green_fruits
# }

# # for track_id in [6,7,12,28]:
# cam = video_wrapper(args.movie_path, args.rotate, args.depth_minimum, args.depth_maximum)
# for fruit_color, track_ids in fruit_dict.items():
#     for track_id in track_ids:
#         frame_numbers = get_track_id_frames(measures_frame, track_id)
#         boxes = get_track_id_boxes(measures_frame, track_id)

#         for i in tqdm(range(len(frame_numbers))):
#             frame, box = frame_numbers[i], boxes[i]
#             rgb_img, _, pc_img = cam.get_zed(frame)
#             x1,y1,x2,y2 = validate_bbox(box, rgb_img)
#             pc_crop = pc_img[y1:y2, x1:x2,:3]
#             rgb_crop = rgb_img[y1:y2, x1:x2, ::-1]
#             hsv = cv2.cvtColor(rgb_crop, cv2.COLOR_RGB2HSV)
#             hue, sat, val = cv2.split(hsv.copy())
#         mean_hue, median_hue = np.nanmean(hue), np.nanmedian(hue)
#         hist_vals, hist_bins = np.histogram(hue, bins = 50)
#         mode_hue = hist_bins[np.argmax(hist_vals)]
#         title = f"t_id: {track_id}, fruit color: {fruit_color}, mean hue:{mean_hue} , median hue:{median_hue}, mode hue:{mode_hue}"
#         plot_2_imgs(rgb_crop, hue, title)
# cam.close()

In [ ]:
53, 163, 471, 1112, 1910
track_id = 227
cam = video_wrapper(args.movie_path, args.rotate, args.depth_minimum, args.depth_maximum)
frame_numbers = get_track_id_frames(measures_frame, track_id)
i = len(frame_numbers) -1
frame, box = frame_numbers[i], boxes[i]
rgb_img, _, pc_img = cam.get_zed(frame)
x1,y1,x2,y2 = validate_bbox(box, rgb_img)
pc_crop = pc_img[y1:y2, x1:x2,:3]
plt.imshow(rgb_img[y1:y2, x1:x2, ::-1])
plt.title(track_id)
plt.show()
cam.close()

In [ ]:
def get_color(rgb_crop):
    if not len(rgb_crop):
        return (255,255,255)
    hsv = cv2.cvtColor(rgb_crop.astype(np.uint8), cv2.COLOR_RGB2HSV)
    hue, sat, val = cv2.split(hsv.copy())
    mean_hue, median_hue = np.nanmean(hue), np.nanmedian(hue)
    hist_vals, hist_bins = np.histogram(hue, bins = 100)
    mode_hue = hist_bins[np.argmax(hist_vals)]
    if mode_hue < 10:
        return (255,0,0)
    if mode_hue < 17.5:
        return (255,125,0)
    if mode_hue < 45:
        w, h = rgb_crop.shape[:2]
        w_025 = int(w/4)
        h_025 = int(h/4)
        hue_cut = hue[h_025:-h_025,w_025:-w_025]
        mean_hue_cut, median_hue_cut = np.nanmean(hue_cut), np.nanmedian(hue_cut)
        if median_hue_cut * 1.15 > mean_hue_cut: # low "skew"
            return (255,255,0)
        else:
            return (128,0,128)
    return (0,255,0)

track_id = 1443
cam = video_wrapper(args.movie_path, args.rotate, args.depth_minimum, args.depth_maximum)
frame_numbers = get_track_id_frames(measures_frame, track_id)
boxes = get_track_id_boxes(measures_frame, track_id)
i = 7
for i in range(len(frame_numbers)):
    frame, box = frame_numbers[i], boxes[i]
    rgb_img, _, pc_img = cam.get_zed(frame)
    x1,y1,x2,y2 = validate_bbox(box, rgb_img)
    pc_crop = pc_img[y1:y2, x1:x2,:3]
    rgb_crop = rgb_img[y1:y2, x1:x2, ::-1]
    rgb_c = rgb_crop.copy()
    w, h = rgb_crop.shape[:2]
    w_025 = int(w/4)
    h_025 = int(h/4)
    hsv = cv2.cvtColor(rgb_crop, cv2.COLOR_RGB2HSV)
    hue, sat, val = cv2.split(hsv.copy())
    mean_hue, median_hue = np.nanmean(hue), np.nanmedian(hue)
    hist_vals, hist_bins = np.histogram(hue, bins = 100)
    mode_hue = hist_bins[np.argmax(hist_vals)]
    hue_cut = hue[h_025:-h_025,w_025:-w_025]
    mean_hue_cut, median_hue_cut = np.nanmean(hue_cut), np.nanmedian(hue_cut)
    hist_vals_cut, hist_bins_cut = np.histogram(hue_cut, bins = 100)
    mode_hue_cut = hist_bins_cut[np.argmax(hist_vals_cut)]
    
    color = get_color(rgb_crop)
    rgb_crop[0,:] = color
    rgb_crop[-1,:] = color
    rgb_crop[:,0] = color
    rgb_crop[:,-1] = color
    print(f"""mean:, {mean_hue}, median: {median_hue}, mode: {mode_hue}, std: {np.nanstd(hue)} \n
          mean:, {mean_hue_cut}, median: {median_hue_cut}, mode: {mode_hue_cut}, std: {np.nanstd(hue_cut)}""")
    plt.hist(hue.flatten(), bins = 50,density = True)
    plt.show()
    plt.imshow(rgb_crop)
    plt.title(frame)
    plt.show()

In [ ]:
cut_box_pc = cut_center_of_box(pc_crop, 0.1)
cut_box_rgb = cut_center_of_box(rgb_img[y1:y2, x1:x2], 0.1)
plot_2_imgs(cut_box_pc[:, :, 2], cut_box_rgb)

In [ ]:
def apply_sobol(det_crop, plot_change=False):
    """
    applies sobol filterning on image
    :param det_crop: image to apply filter on
    :param plot_change: flag to show the image after applying sobol
    :return: image after sobol filtering
    """
    torch_img = K.utils.image_to_tensor(det_crop)
    torch_img = torch_img[None, ...].float() / 255.
    torch_img = K.enhance.adjust_contrast(torch_img, 0.5)
    torch_img_gray = K.color.rgb_to_grayscale(torch_img)
    processed_img = K.filters.sobel(torch_img_gray, True, 1e-3) 
    if plot_change:
        plot_2_imgs(det_crop, processed_img.detach().numpy()[0, 0] > 0.05)
    return processed_img.detach().numpy()[0, 0]
out_img = apply_sobol(rgb_img[y1:y2, x1:x2, ::-1], True)

In [ ]:
min_area = 5
out_img = out_img > 0.04
rgb_crop = rgb_img[y1:y2, x1:x2, ::-1].copy()
contours, _ = cv2.findContours(out_img.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

filtered_contours = []
for c in contours:
    if cv2.contourArea(c) >= min_area:
        filtered_contours.append(c)

mask = np.zeros_like(out_img).astype(np.uint8)
hull = cv2.convexHull(np.concatenate(filtered_contours))
cv2.drawContours(mask, [hull], 0, (255, 255, 255), -1)
plot_2_imgs(out_img, mask)

rgb_c = rgb_crop.copy()
rgb_c[np.logical_not(mask)] = 0
plot_2_imgs(rgb_crop, rgb_c)

In [ ]:
plot_2_imgs(rgb_crop, hue_filtering(rgb_crop))

In [ ]:
hist_bins[np.argmax(hist_vals)] + 1*np.std(hue)

In [ ]:
hsv = cv2.cvtColor(rgb_c, cv2.COLOR_RGB2HSV)
hue, sat, v = cv2.split(hsv.copy())
rgb_crop = rgb_c.copy()
plot_2_imgs(rgb_c, hue)
rgb_crop[hue > 200] = 0
plot_2_imgs(rgb_c, rgb_crop)
rgb_crop[hue > 150] = 0
plot_2_imgs(rgb_c, rgb_crop)
rgb_crop[hue > 100] = 0
plot_2_imgs(rgb_c, rgb_crop)
rgb_crop[hue > 50] = 0
plot_2_imgs(rgb_c, rgb_crop)
rgb_crop[hue > 25] = 0
plot_2_imgs(rgb_c, rgb_crop)